## Summary

To apply random forest technique from the h2o package to the problem of predicting customer value.

In [36]:
import time

import sys
sys.path.append('../../common_routines/')

from relevant_functions import (get_train_data,
                                get_test_data,
                                get_all_predictor_cols,
                                get_rel_cols)


import h2o
import numpy as np

In [6]:
INPUT_DIR = '../../input/'

In [7]:
ts = time.time()
train = get_train_data(INPUT_DIR)
time.time() - ts

5.172586917877197

In [8]:
# Remove unnecessary columns to prevent confusion.
train.drop(columns=['ID', 'target'], inplace=True)

### Start an h2o instance

We need to do that before we can use the package.

In [4]:
h2o.init(nthreads=-1, max_mem_size=15)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.5" 2019-10-15 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.5+10-LTS); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.5+10-LTS, mixed mode)
  Starting server from /Users/babs4JESUS/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/cz/3nvpl4mj0g5ds3hlsc15wxdr0000gn/T/tmp9a28d9au
  JVM stdout: /var/folders/cz/3nvpl4mj0g5ds3hlsc15wxdr0000gn/T/tmp9a28d9au/h2o_babs4JESUS_started_from_python.out
  JVM stderr: /var/folders/cz/3nvpl4mj0g5ds3hlsc15wxdr0000gn/T/tmp9a28d9au/h2o_babs4JESUS_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_babs4JESUS_eey2rp
H2O cluster total nodes:,1
H2O cluster free memory:,15 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [10]:
ts = time.time()
train_h2o = h2o.H2OFrame(train)
time.time() - ts

Parse progress: |█████████████████████████████████████████████████████████| 100%


15.268220901489258

### Check results of cross validation.

Let us see how things look !

In [11]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [15]:
X_COLUMNS = get_all_predictor_cols(train)
Y_COLUMN = 'log_target'

In [17]:
ts = time.time()
cross_val_model = H2ORandomForestEstimator(model_id='cross_val_model', seed=1, nfolds=5)
cross_val_model.train(x=X_COLUMNS, y= Y_COLUMN, training_frame=train_h2o)
time.time() - ts

drf Model Build progress: |███████████████████████████████████████████████| 100%


2702.940629720688

In [18]:
cross_val_model

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  cross_val_model


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,465592.0,20.0,20.0,20.0,591.0,881.0,728.6




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 2.385675472027168
RMSE: 1.5445631978093897
MAE: 1.241445135355738
RMSLE: 0.10388075199188072
Mean Residual Deviance: 2.385675472027168

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 2.3747882338588546
RMSE: 1.5410347932019104
MAE: 1.2358036492110958
RMSLE: 0.10370428551260114
Mean Residual Deviance: 2.3747882338588546

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,1.2359084,0.023778666,1.2333033,1.2080435,1.2704134,1.2457173,1.2220647
1,mean_residual_deviance,2.3750353,0.07138295,2.4314992,2.271778,2.4517794,2.3699005,2.3502188
2,mse,2.3750353,0.07138295,2.4314992,2.271778,2.4517794,2.3699005,2.3502188
3,r2,0.22418003,0.02238349,0.1901805,0.22689798,0.23591278,0.2499543,0.21795453
4,residual_deviance,2.3750353,0.07138295,2.4314992,2.271778,2.4517794,2.3699005,2.3502188
5,rmse,1.540975,0.023221916,1.5593265,1.5072418,1.5658159,1.5394481,1.5330423
6,rmsle,0.10370939,0.0018986115,0.104609214,0.10051033,0.105381764,0.10445981,0.10358586



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2019-11-25 13:22:49,37 min 22.889 sec,0.0,NaN,NaN,NaN
1,,2019-11-25 13:22:58,37 min 32.145 sec,1.0,1.755027,1.387724,3.080121
2,,2019-11-25 13:23:07,37 min 40.588 sec,2.0,1.788884,1.406125,3.200104
3,,2019-11-25 13:23:14,37 min 47.828 sec,3.0,1.759264,1.390839,3.095012
4,,2019-11-25 13:23:22,37 min 55.967 sec,4.0,1.747639,1.375627,3.054242
5,,2019-11-25 13:23:32,38 min 5.775 sec,5.0,1.724628,1.363064,2.974341
6,,2019-11-25 13:23:44,38 min 17.350 sec,6.0,1.704177,1.353991,2.904218
7,,2019-11-25 13:23:53,38 min 26.439 sec,7.0,1.689985,1.345845,2.856051
8,,2019-11-25 13:24:04,38 min 37.659 sec,8.0,1.676558,1.334697,2.810848
9,,2019-11-25 13:24:13,38 min 46.762 sec,9.0,1.663072,1.325094,2.765807



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,f190486d6,25280.314453,1.000000,0.111545
1,58e2e02e6,13369.619141,0.528855,0.058991
2,eeb9cd3aa,9913.291016,0.392135,0.043741
3,9fd594eec,7249.024902,0.286746,0.031985
4,15ace8c9f,4221.983887,0.167007,0.018629
5,6eef030c1,3434.542725,0.135858,0.015154
6,994b4c2ac,1984.114990,0.078485,0.008755
7,f514fdb2e,1758.339478,0.069554,0.007758
8,20aa07010,1607.534058,0.063588,0.007093
9,b43a7cfd5,1520.040527,0.060127,0.006707



See the whole table with table.as_data_frame()


In [20]:
ts = time.time()
cross_val_model = H2ORandomForestEstimator(model_id='cross_val_model', seed=1, nfolds=5, ntrees=150)
cross_val_model.train(x=X_COLUMNS, y=Y_COLUMN, training_frame=train_h2o)
time.time() - ts

drf Model Build progress: |███████████████████████████████████████████████| 100%


11385.765273809433

In [21]:
cross_val_model

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  cross_val_model


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,150.0,150.0,1388584.0,20.0,20.0,20.0,499.0,948.0,724.37335




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 2.351116845606134
RMSE: 1.5333352032762224
MAE: 1.2312403424493086
RMSLE: 0.10323195513230768
Mean Residual Deviance: 2.351116845606134

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 2.3655790157561714
RMSE: 1.5380438926624205
MAE: 1.2354670678431245
RMSLE: 0.10353724870051666
Mean Residual Deviance: 2.3655790157561714

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,1.2355988,0.025489954,1.2317092,1.2073971,1.2750363,1.2420793,1.2217724
1,mean_residual_deviance,2.3657188,0.06962698,2.410596,2.2705903,2.4501874,2.3660457,2.331175
2,mse,2.3657188,0.06962698,2.410596,2.2705903,2.4501874,2.3660457,2.331175
3,r2,0.22726385,0.019820923,0.19714247,0.2273022,0.23640895,0.25117427,0.2242914
4,residual_deviance,2.3657188,0.06962698,2.410596,2.2705903,2.4501874,2.3660457,2.331175
5,rmse,1.5379558,0.022664055,1.5526093,1.5068479,1.5653074,1.5381956,1.5268186
6,rmsle,0.10353986,0.0018120669,0.10417351,0.10061785,0.10535577,0.10440282,0.10314936



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2019-11-26 14:35:28,2:43:18.470,0.0,NaN,NaN,NaN
1,,2019-11-26 14:35:36,2:43:27.212,1.0,1.755027,1.387724,3.080121
2,,2019-11-26 14:35:44,2:43:34.787,2.0,1.788884,1.406125,3.200104
3,,2019-11-26 14:35:51,2:43:41.952,3.0,1.759264,1.390839,3.095012
4,,2019-11-26 14:35:59,2:43:49.979,4.0,1.747639,1.375627,3.054242
5,,2019-11-26 14:36:07,2:43:57.629,5.0,1.724628,1.363064,2.974341
6,,2019-11-26 14:36:15,2:44:06.068,6.0,1.704177,1.353991,2.904218
7,,2019-11-26 14:36:24,2:44:14.919,7.0,1.689985,1.345845,2.856051
8,,2019-11-26 14:36:32,2:44:23.246,8.0,1.676558,1.334697,2.810848
9,,2019-11-26 14:36:42,2:44:32.722,9.0,1.663072,1.325094,2.765807



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,f190486d6,68977.367188,1.000000,0.102355
1,58e2e02e6,38143.957031,0.552992,0.056601
2,eeb9cd3aa,27366.585938,0.396747,0.040609
3,9fd594eec,23966.115234,0.347449,0.035563
4,6eef030c1,11122.238281,0.161245,0.016504
5,15ace8c9f,10976.938477,0.159138,0.016289
6,20aa07010,8615.379883,0.124902,0.012784
7,58232a6fb,5556.180176,0.080551,0.008245
8,491b9ee45,4808.947754,0.069718,0.007136
9,f514fdb2e,4773.053711,0.069197,0.007083



See the whole table with table.as_data_frame()


### Train model over the entire data set

Let us train model over the entire data set and generate predictions on the test data.

In [22]:
ts = time.time()
model_full_data = H2ORandomForestEstimator(model_id='model_full_data', seed=1, ntrees=150, max_depth=150)
model_full_data.train(x=X_COLUMNS, y=Y_COLUMN, training_frame=train_h2o)
time.time() - ts

drf Model Build progress: |███████████████████████████████████████████████| 100%


24464.621030807495

In [23]:
ts = time.time()
test = get_test_data(INPUT_DIR)
time.time() - ts

75.0000319480896

In [27]:
ts = time.time()
test_prediction_out = model_full_data.predict(h2o.H2OFrame(test[X_COLUMNS]))


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


NameError: name 'test_log_predictions' is not defined

In [37]:
ts = time.time()
test_log_predictions = test_prediction_out.as_data_frame()['predict'].values.tolist()
test_log_predictions  = [x if x > 0 else 0 for x in test_log_predictions]
test['target'] = np.exp(test_log_predictions) - 1.0
time.time() - ts

0.17999911308288574

In [38]:
test[['ID', 'target']].to_csv('submission_random_forest_h2o.csv', index=False)